In [8]:
import pyspark.sql.functions as F

StatementMeta(, cd199acf-0209-4be1-9dc1-f50f6b51e5e6, 10, Finished, Available)

In [9]:
old_order = spark.sql("""SELECT MAX(dd.date) as max_date FROM DQT_Gold_LakeHouse_Layer.fact_sales fs  

JOIN  DQT_Gold_LakeHouse_Layer.dim_date dd ON 
fs.order_date_key = dd.date_key
WHERE dd.year = 2018
""")


max_date = old_order.collect()[0]['max_date']


StatementMeta(, cd199acf-0209-4be1-9dc1-f50f6b51e5e6, 11, Finished, Available)

In [10]:
max_date

StatementMeta(, cd199acf-0209-4be1-9dc1-f50f6b51e5e6, 12, Finished, Available)

datetime.date(2018, 1, 31)

In [11]:
path= "abfss://4155d96d-1328-43ad-982f-7aecb1ee2008@onelake.dfs.fabric.microsoft.com/900594c3-14cd-4919-8406-174ac04587bb/Files/"


orders = spark.read.option("header", "true") .csv(path + "orders")


orders = orders.filter(F.col("order_date_dateorders") > max_date)




StatementMeta(, cd199acf-0209-4be1-9dc1-f50f6b51e5e6, 13, Finished, Available)

In [12]:
from pyspark.sql.functions import col, broadcast

StatementMeta(, cd199acf-0209-4be1-9dc1-f50f6b51e5e6, 14, Finished, Available)

# Load data for mapping

In [13]:
dim_delivery_status = spark.sql("SELECT * FROM DQT_Gold_LakeHouse_Layer.dim_delivery_status")


StatementMeta(, cd199acf-0209-4be1-9dc1-f50f6b51e5e6, 15, Finished, Available)

In [14]:
dim_delivery_risk = spark.sql("SELECT * FROM DQT_Gold_LakeHouse_Layer.dim_delivery_risk LIMIT 1000")


StatementMeta(, cd199acf-0209-4be1-9dc1-f50f6b51e5e6, 16, Finished, Available)

In [15]:
dim_order_status = spark.sql("SELECT * FROM DQT_Gold_LakeHouse_Layer.dim_order_status LIMIT 1000")


StatementMeta(, cd199acf-0209-4be1-9dc1-f50f6b51e5e6, 17, Finished, Available)

In [16]:
dim_shipping_mode = spark.sql("SELECT * FROM DQT_Gold_LakeHouse_Layer.dim_shipping_mode LIMIT 1000")

StatementMeta(, cd199acf-0209-4be1-9dc1-f50f6b51e5e6, 18, Finished, Available)

In [17]:
dim_transaction = spark.sql("SELECT * FROM DQT_Gold_LakeHouse_Layer.dim_transaction LIMIT 1000")

StatementMeta(, cd199acf-0209-4be1-9dc1-f50f6b51e5e6, 19, Finished, Available)

In [18]:
dim_territory = spark.sql("SELECT * FROM DQT_Gold_LakeHouse_Layer.dim_territory")


StatementMeta(, cd199acf-0209-4be1-9dc1-f50f6b51e5e6, 20, Finished, Available)

In [19]:
dim_store = spark.sql("SELECT * FROM DQT_Gold_LakeHouse_Layer.dim_store")


StatementMeta(, cd199acf-0209-4be1-9dc1-f50f6b51e5e6, 21, Finished, Available)

In [20]:
dim_customer = spark.sql("SELECT customer_key, customer_id FROM DQT_Gold_LakeHouse_Layer.dim_customer ")


StatementMeta(, cd199acf-0209-4be1-9dc1-f50f6b51e5e6, 22, Finished, Available)

In [21]:
dim_product = spark.sql("SELECT product_key,product_card_id FROM DQT_Gold_LakeHouse_Layer.dim_product")


StatementMeta(, cd199acf-0209-4be1-9dc1-f50f6b51e5e6, 23, Finished, Available)

# Mapping for orders

## order_status_key

In [22]:
dim_order_status = broadcast(dim_order_status)
orders = orders.withColumnRenamed("order_status", "order_status_t1")

orders = orders.join(dim_order_status, orders['order_status_t1'] == dim_order_status['order_status'],"inner")

orders = orders.drop("order_status_t1", "order_status")

StatementMeta(, cd199acf-0209-4be1-9dc1-f50f6b51e5e6, 24, Finished, Available)

## delivery_status_key

In [23]:
# Broadcast dim_delivery_status DataFrame
dim_delivery_status = broadcast(dim_delivery_status)

#renamed
orders = orders.withColumnRenamed("delivery_status", "delivery_status_t1")

# Join orders with dim_delivery_status
orders = orders.join(dim_delivery_status, orders['delivery_status_t1'] == dim_delivery_status['delivery_status'],"inner")


orders = orders.drop("delivery_status", "delivery_status_t1")

StatementMeta(, cd199acf-0209-4be1-9dc1-f50f6b51e5e6, 25, Finished, Available)

## transaction_key

In [24]:
# Broadcast dim_transaction DataFrame
dim_transaction = broadcast(dim_transaction)

#renamed
orders = orders.withColumnRenamed("type_of_transaction", "type_of_transaction_t1")

# Join orders with dim_transaction
orders = orders.join(dim_transaction, orders['type_of_transaction_t1'] == dim_transaction['type_of_transaction'],"inner")

orders = orders.drop("type_of_transaction", "type_of_transaction_t1")



StatementMeta(, cd199acf-0209-4be1-9dc1-f50f6b51e5e6, 26, Finished, Available)

## shipping_mode_key

In [25]:
# Broadcast dim_dim_shipping_mode DataFrame
dim_shipping_mode = broadcast(dim_shipping_mode)

#renamed
orders = orders.withColumnRenamed("shipping_mode", "shipping_mode_t1")

# Join orders with dim_dim_shipping_mode
orders = orders.join(dim_shipping_mode, orders['shipping_mode_t1'] == dim_shipping_mode['shipping_mode'],"inner")

orders = orders.drop("shipping_mode", "shipping_mode_t1")


StatementMeta(, cd199acf-0209-4be1-9dc1-f50f6b51e5e6, 27, Finished, Available)

## customer

In [26]:
orders = orders.withColumnRenamed("customer_id", "customer_id_t1")
orders = orders.join(dim_customer, dim_customer['customer_id'] == orders['customer_id_t1'], "inner")
orders = orders.drop("customer_id_t1", "customer_id")



StatementMeta(, cd199acf-0209-4be1-9dc1-f50f6b51e5e6, 28, Finished, Available)

## product

In [27]:
orders = orders.withColumnRenamed("product_card_id", "product_card_id_t1")
orders = orders.join(dim_product , dim_product.product_card_id == orders.product_card_id_t1, "inner")
orders = orders.drop("product_card_id", "product_card_id_t1")




StatementMeta(, cd199acf-0209-4be1-9dc1-f50f6b51e5e6, 29, Finished, Available)

## dim_territory

In [28]:


# Join orders with dim_dim_shipping_mode
orders = orders.join(dim_territory, orders['concat_territory'] == dim_territory['ter_concat_territory'],"inner")

orders = orders.drop("concat_territory", "ter_concat_territory","market", "order_region", "order_state", "order_country", "order_city")


StatementMeta(, cd199acf-0209-4be1-9dc1-f50f6b51e5e6, 30, Finished, Available)

## dim_store_region

In [29]:


# Join orders with dim_dim_shipping_mode
orders = orders.join(dim_store, orders['concat_store_address'] == dim_store['st_concat_store_address'],"inner")

orders = orders.drop("st_concat_store_address", "concat_store_address","store_country", "store_city","store_street", "longitude", "latitude")



StatementMeta(, cd199acf-0209-4be1-9dc1-f50f6b51e5e6, 31, Finished, Available)

## format date orders and shipping

In [30]:
from pyspark.sql.functions import to_timestamp, date_format

StatementMeta(, cd199acf-0209-4be1-9dc1-f50f6b51e5e6, 32, Finished, Available)

In [31]:
orders = orders.withColumn("order_time_key", 
date_format(to_timestamp("order_date_dateorders", "yyyy-MM-dd'T'HH:mm:ss.SSS'Z'"), "HHmm")
)

StatementMeta(, cd199acf-0209-4be1-9dc1-f50f6b51e5e6, 33, Finished, Available)

In [32]:
orders = orders.withColumn("shipping_time_key",

date_format(to_timestamp("shipping_date_dateorders", "yyyy-MM-dd'T'HH:mm:ss.SSS'Z'"), "HHmm")
)

StatementMeta(, cd199acf-0209-4be1-9dc1-f50f6b51e5e6, 34, Finished, Available)

In [33]:
orders = orders.withColumn("shipping_date_dateorders", 
                   date_format(to_timestamp("shipping_date_dateorders", "yyyy-MM-dd'T'HH:mm:ss.SSS'Z'"), "yyyyMMdd"))
orders = orders.withColumnRenamed("shipping_date_dateorders", "shipping_date_key")


StatementMeta(, cd199acf-0209-4be1-9dc1-f50f6b51e5e6, 35, Finished, Available)

In [34]:
orders = orders.withColumn("order_date_dateorders", 
                   date_format(to_timestamp("order_date_dateorders", "yyyy-MM-dd'T'HH:mm:ss.SSS'Z'"), "yyyyMMdd"))
orders = orders.withColumnRenamed("order_date_dateorders", "order_date_key")



StatementMeta(, cd199acf-0209-4be1-9dc1-f50f6b51e5e6, 36, Finished, Available)

## format late_develiry_risk

In [35]:
from pyspark.sql.functions import when

StatementMeta(, cd199acf-0209-4be1-9dc1-f50f6b51e5e6, 37, Finished, Available)

In [36]:
orders = orders.withColumn("late_delivery_risk",when(col("late_delivery_risk") == '0', '2').otherwise('1'))
orders = orders.withColumnRenamed("late_delivery_risk","delivery_risk_key")
orders = orders.withColumn("delivery_risk_key",col("delivery_risk_key").cast("int") )


StatementMeta(, cd199acf-0209-4be1-9dc1-f50f6b51e5e6, 38, Finished, Available)

# Create fact

In [37]:
orders.createOrReplaceTempView("orders")

StatementMeta(, cd199acf-0209-4be1-9dc1-f50f6b51e5e6, 39, Finished, Available)

## fact_sales

In [38]:
orders.dtypes

StatementMeta(, cd199acf-0209-4be1-9dc1-f50f6b51e5e6, 40, Finished, Available)

[('order_id', 'string'),
 ('order_date_key', 'string'),
 ('shipping_date_key', 'string'),
 ('delivery_risk_key', 'int'),
 ('sales', 'string'),
 ('benefit_per_order', 'string'),
 ('order_item_discount', 'string'),
 ('order_item_quantity', 'string'),
 ('order_profit_per_order', 'string'),
 ('year', 'int'),
 ('month', 'int'),
 ('order_status_key', 'int'),
 ('delivery_status_key', 'int'),
 ('transaction_key', 'int'),
 ('shipping_mode_key', 'int'),
 ('customer_key', 'int'),
 ('product_key', 'int'),
 ('territory_key', 'int'),
 ('store_key', 'int'),
 ('order_time_key', 'string'),
 ('shipping_time_key', 'string')]

In [39]:
fact_sales = spark.sql("""

SELECT order_id, product_key, order_date_key, shipping_date_key, delivery_risk_key, sales, 
order_status_key, delivery_status_key, transaction_key, shipping_mode_key, territory_key,
store_key, order_time_key, shipping_time_key, customer_key, 
benefit_per_order, order_item_discount, order_item_quantity FROM orders

""")
display(orders)

StatementMeta(, cd199acf-0209-4be1-9dc1-f50f6b51e5e6, 41, Finished, Available)

SynapseWidget(Synapse.DataFrame, cb2d09db-b2a0-42cf-a034-0ec48b4f4e2e)

In [40]:
fact_sales= fact_sales.select(
    col("order_id").cast("string"),
    col("product_key").cast("int"),
    col("order_date_key").cast("int"),
    col("order_time_key").cast("int"),
    col("shipping_date_key").cast("int"),
    col("shipping_time_key").cast("int"),
    col("delivery_risk_key").cast("int"),
    col("customer_key").cast("int"),
    col("sales").cast("float"),
    col("order_status_key").cast("int"),
    col("delivery_status_key").cast("int"),
    col("transaction_key").cast("int"),
    col("shipping_mode_key").cast("int"),
    col("territory_key").cast("int"),
    col("store_key").cast("int"),
    col("benefit_per_order").cast("float"),
    col("order_item_discount").cast("float"),
    col("order_item_quantity").cast("int")
)


StatementMeta(, cd199acf-0209-4be1-9dc1-f50f6b51e5e6, 42, Finished, Available)

## fact_monthly_kpi

In [90]:
fact_monthly_kpi = spark.sql("""
  WITH  delivery_status_4_cte AS (
        SELECT 
            dd.year, 
            dd.month, 
            o.store_key, 
            COUNT(delivery_status_key) AS on_time_delivery
        FROM 
            (SELECT 
                order_id, delivery_status_key, order_date_key, store_key 
            FROM 
                orders 
            GROUP BY 
                order_id, delivery_status_key, order_date_key, store_key
            ) AS o
        JOIN 
            dim_date dd 
        ON 
            o.order_date_key = dd.date_key
        WHERE 
            delivery_status_key = 4 
        GROUP BY 
            dd.year, dd.month, o.store_key
    ),

    total_cte AS (
        SELECT 
            dd.year, 
            dd.month, 
            o.store_key, 
            COUNT(o.delivery_status_key) AS total_delivery_status,
            SUM(o.total_sales) AS actual_sales,
            SUM(o.total_profit) AS actual_profit
        FROM 
            (SELECT 
                order_id, 
                delivery_status_key, 
                customer_key, 
                order_date_key, 
                store_key, 
                SUM(sales) AS total_sales ,
                SUM(benefit_per_order) AS total_profit
            FROM 
                orders 
            GROUP BY 
                order_id, 
                delivery_status_key, 
                order_date_key, 
                customer_key, 
                store_key
            ) AS o 
        JOIN 
            dim_date dd ON o.order_date_key = dd.date_key 
        GROUP BY 
            dd.year, dd.month, o.store_key
    ),
    total_cus AS (
            SELECT 
                dd.year,
                dd.month,
                store_key,
                COUNT(DISTINCT o.customer_key) AS total_customer
            FROM 
                orders o
            JOIN 
                dim_date dd ON dd.date_key = o.order_date_key
            GROUP BY 
                dd.year,dd.month, store_key
    ),

    cus_cte AS (
            SELECT 
                year, 
                month, 
                store_key, 
                SUM(CASE WHEN num_orders_in_month >= 2 THEN 1 ELSE 0 END) as num_cus_return
            FROM (
                SELECT 
                    dd.year,
                    dd.month,
                    o.customer_key,
                    o.store_key,
                    COUNT(DISTINCT o.order_id) AS num_orders_in_month
                FROM 
                    dim_date dd
                RIGHT JOIN 
                    orders o ON dd.date_key = o.order_date_key AND o.order_date_key <= (SELECT MAX(order_date_key) FROM orders)
                GROUP BY 
                    dd.year, dd.month, o.customer_key, o.store_key
            ) as subquery
            GROUP BY 
                year, month, store_key
    )

SELECT 
    date_format(last_day(to_date(concat(ds.year, '-', ds.month, '-', '01'))), 'yyyyMMdd') AS date_key,
    ds.store_key, 
    ds.on_time_delivery,
    tc.total_delivery_status, 
    cc.num_cus_return, 
    tcus.total_customer, 
    tc.actual_sales, 
    tc.actual_profit
  

FROM 
    delivery_status_4_cte ds 
JOIN
    total_cte tc ON ds.year = tc.year AND ds.month = tc.month AND ds.store_key = tc.store_key
JOIN 
    cus_cte cc ON ds.year = cc.year AND ds.month = cc.month AND ds.store_key = cc.store_key
JOIN total_cus tcus ON tcus.year  = cc.year AND tcus.month  = cc.month AND tcus.store_key = cc.store_key

""")


StatementMeta(, cd199acf-0209-4be1-9dc1-f50f6b51e5e6, 92, Finished, Available)

In [55]:
fact_monthly_kpi = fact_monthly_kpi.withColumn("date_key", col("date_key").cast("int"))

StatementMeta(, cd199acf-0209-4be1-9dc1-f50f6b51e5e6, 57, Finished, Available)

## fact_delivery_performance

In [43]:
fact_weekly_delivery_performance = spark.sql("""
WITH cte AS (
    SELECT
        d.year, 
        d.month, 
        o.order_time_key,

        o.territory_key,
        o.order_date_key,

        SUM(CASE WHEN o.delivery_status_key = 3 THEN 1 ELSE 0 END) AS total_cancellation,
        COUNT(o.order_id) AS total_orders,
        SUM(CASE WHEN o.delivery_status_key = 2 THEN 1 ELSE 0 END) AS late_orders,
        SUM(CASE WHEN o.delivery_status_key = 4 THEN 1 ELSE 0 END) AS on_time_orders, 
        SUM(CASE WHEN o.delivery_status_key = 1 THEN 1 ELSE 0 END) AS advanced_orders 
    FROM
        (SELECT order_date_key, order_time_key, delivery_status_key, order_id, territory_key
         FROM orders
         GROUP BY order_date_key, order_time_key, delivery_status_key, order_id, territory_key) o
    JOIN
        dim_date d ON o.order_date_key = d.date_key
    GROUP BY
        o.territory_key, d.year, d.month, o.order_date_key, o.order_time_key
)
SELECT 
    order_date_key,
    order_time_key,
    territory_key,
    total_cancellation,
    on_time_orders, 
    late_orders,
    advanced_orders,
    total_orders
FROM cte;


""")

StatementMeta(, cd199acf-0209-4be1-9dc1-f50f6b51e5e6, 45, Finished, Available)

In [45]:
fact_weekly_delivery_performance = fact_weekly_delivery_performance.withColumn("order_date_key", col("order_date_key").cast("int"))\
.withColumn("order_time_key", col("order_time_key").cast("int"))
fact_weekly_delivery_performance.dtypes

StatementMeta(, cd199acf-0209-4be1-9dc1-f50f6b51e5e6, 47, Finished, Available)

[('order_date_key', 'int'),
 ('order_time_key', 'int'),
 ('territory_key', 'int'),
 ('total_cancellation', 'bigint'),
 ('on_time_orders', 'bigint'),
 ('late_orders', 'bigint'),
 ('advanced_orders', 'bigint'),
 ('total_orders', 'bigint')]

# fact_sales

In [ ]:
fact_sales.write.mode("append").format("delta").saveAsTable("fact_sales")

StatementMeta(, , , Cancelled, )

In [ ]:
fact_monthly_kpi.write.mode("append").format("delta").saveAsTable("fact_monthly_kpi")

StatementMeta(, , , Cancelled, )

In [ ]:
fact_weekly_delivery_performance.write.mode("append").format("delta").saveAsTable("fact_weekly_delivery_performance")

StatementMeta(, , , Cancelled, )